<a href="https://colab.research.google.com/github/aivc24024/lenet-images/blob/main/numberofthetrees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
!pip install tensorflow opencv-python numpy matplotlib rasterio
!pip install scikit-learn

In [79]:
import cv2
import numpy as np

# Load a satellite image
image = cv2.imread('olive trees.png', cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

# Normalize pixel values (0-1)
image = image.astype('float32') / 255.0

# Resize if needed (e.g., 256x256 for a CNN)
image = cv2.resize(image, (256, 256))

In [80]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

In [81]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1)  # Output: predicted count of objects
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Regression problem

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [82]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False  # Freeze layers

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1)  # Output: count
])

model.compile(optimizer='adam', loss='mse')

In [87]:
import numpy as np
# ... (your previous code to create patches and counts)

# Example: Assuming X contains your patches and y contains object counts
# Split into training and testing sets (adjust ratio as needed)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Assuming X_train contains images and y_train contains counts
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[
        EarlyStopping(patience=5),
        ModelCheckpoint('best_model.h5', save_best_only=True)
    ]
)


Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 35.8955

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 35.8955 - val_loss: 10.3249
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 34.2174

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 34.2174 - val_loss: 9.3964
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 32.9189

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 32.9189 - val_loss: 8.9907
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 31.9234

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 31.9234 - val_loss: 8.9788
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 31.1682 - val_loss: 9.2536


In [88]:
# Predict counts on test data
predictions = model.predict(X_test)

# Compare with ground truth
print("Mean Absolute Error (MAE):", np.mean(np.abs(predictions - y_test)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 702ms/step
Mean Absolute Error (MAE): 3.0


In [54]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('olive trees.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

# Define patch size and stride
patch_size = 256
stride = 256  # Non-overlapping patches (adjust for overlap if needed)

# Split into patches
patches = []
for y in range(0, image.shape[0], stride):
    for x in range(0, image.shape[1], stride):
        patch = image[y:y+patch_size, x:x+patch_size]
        if patch.shape[0] == patch_size and patch.shape[1] == patch_size:
            patches.append(patch)

X = np.array(patches)  # Shape: (num_patches, 256, 256, 3)

In [86]:
# Example: Manually counted objects for 4 patches
y = np.array([3, 3, 4, 0,2,9, 12, 13])  # Replace with your actual counts

In [66]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generate augmented data
augmented_images = []
augmented_counts = []
for i in range(5):
    img = X[i]  # Use the current index for X
    count = y[i]  # Use the current index for y
    for _ in range(10):  # Generate 10 augmented versions per patch
        augmented = datagen.random_transform(img)
        augmented_images.append(augmented)
        augmented_counts.append(count)

X_augmented = np.array(augmented_images)
y_augmented = np.array(augmented_counts)

In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)  # Output: predicted count
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = model.fit(
    X_augmented, y_augmented,
    epochs=5,
    batch_size=8,
    validation_split=0.2  # Use 20% of augmented data for validation
)

Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - loss: 18561144.0000 - mae: 2469.7771 - val_loss: 639645.0000 - val_mae: 799.7535
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 406970.1250 - mae: 563.6488 - val_loss: 25734.7773 - val_mae: 160.4065
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 997ms/step - loss: 22183.8438 - mae: 116.8425 - val_loss: 554.5760 - val_mae: 23.5456
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 918ms/step - loss: 5731.3442 - mae: 53.1613 - val_loss: 6306.0889 - val_mae: 79.4056
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 6307.3945 - mae: 75.6896 - val_loss: 2337.2690 - val_mae: 48.3379


In [68]:
# Predict counts for a new patch
new_patch = cv2.resize(image[300:556, 400:656], (256, 256))  # Example crop
predicted_count = model.predict(np.expand_dims(new_patch, axis=0))[0][0]
print(f"Predicted count: {predicted_count:.0f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Predicted count: 56
